## Preparing the Data

Firstly, we must import the dataset into the notebook

In [2]:
import pandas as pd

# Importing the dataset
train = pd.read_excel('OpArticles_ADUs.xlsx')
test = pd.read_excel('OpArticles.xlsx')

In [3]:
train.head()

,article_id,annotator,node,ranges,tokens,label
0,5d04a31b896a7fea069ef06f,A,0,"[[2516, 2556]]",O facto não é apenas fruto da ignorância,Value
1,5d04a31b896a7fea069ef06f,A,1,"[[2568, 2806]]",havia no seu humor mais jornalismo (mais inves...,Value
2,5d04a31b896a7fea069ef06f,A,3,"[[3169, 3190]]",É tudo cómico na FIFA,Value
3,5d04a31b896a7fea069ef06f,A,4,"[[3198, 3285]]",o que todos nós permitimos que esta organizaçã...,Value
4,5d04a31b896a7fea069ef06f,A,6,"[[4257, 4296]]",não nos fazem rir à custa dos poderosos,Value


In [4]:
test.head()

,article_id,title,authors,body,meta_description,topics,keywords,publish_date,url_canonical
0,5d04a31b896a7fea069ef06f,"Pouco pão e muito circo, morte e bocejo",['José Vítor Malheiros'],"O poeta espanhol António Machado escrevia, uns...","É tudo cómico na FIFA, porque todos os dias a ...",Sports,"['Brasil', 'Campeonato do Mundo', 'Desporto', ...",2014-06-17 00:16:00,https://www.publico.pt/2014/06/17/desporto/opi...
1,5d04a3fc896a7fea069f0717,Portugal nos Mundiais de Futebol de 2010 e 2014,['Rui J. Baptista'],“O mais excelente quadro posto a uma luz logo ...,Deve ser evidenciado o clima favorável criado ...,Sports,"['Brasil', 'Campeonato do Mundo', 'Coreia do N...",2014-07-05 02:46:00,https://www.publico.pt/2014/07/05/desporto/opi...
2,5d04a455896a7fea069f07ab,"Futebol, guerra, religião",['Fernando Belo'],1. As sociedades humanas parecem ser regidas p...,O futebol parece ser um sucedâneo quer da lei ...,Sports,"['A guerra na Síria', 'Desporto', 'Futebol', '...",2014-07-12 16:05:33,https://www.publico.pt/2014/07/12/desporto/opi...
3,5d04a52f896a7fea069f0921,As razões do Qatar para acolher o Mundial em 2022,['Hamad bin Khalifa bin Ahmad Al Thani'],Este foi um Mundial incrível. Vimos actuações ...,Queremos cooperar plenamente com a investigaçã...,Sports,"['Desporto', 'FIFA', 'Futebol', 'Mundial de fu...",2014-07-27 02:00:00,https://www.publico.pt/2014/07/27/desporto/opi...
4,5d04a8d7896a7fea069f6997,A política no campo de futebol,['Carlos Nolasco'],O futebol sempre foi um jogo aparentemente sim...,Retirar a expressão política do futebol é reti...,Sports,"['Albânia', 'Campeonato da Europa', 'Desporto'...",2014-10-23 00:16:00,https://www.publico.pt/2014/10/23/desporto/opi...


## Cleanup and normalization

The next step is to cleanup our dataset and normalize some data

#### Removing non-alphabetic chars

Let's start by removing any non-alpha chars, using a regular expression. We'll create a separate corpus (a list of tokens), so that we leave the original dataset untouched.

#### Lowercasing

We can then apply lowercasing, so that words such as *Amazing*, *AMAZING* and *amazing* all have the same representation.

#### Removing stop words

Another common step which is sometimes applied is to remove any stop words (words that do not have domain semantics attached). We can use the stop words list provided in NLTK for English:

#### Stemming

Finally, we can apply stemming to further reduce the size of the vocabulary through normalization.

In [6]:
import re
from nltk.stem import RSLPStemmer
from nltk.corpus import stopwords

stopwords_list = stopwords.words('portuguese')
stopwords_list.remove('não')

corpus = []
stemmer = RSLPStemmer()
for i in range(0, train['tokens'].size):
    # get review and remove non alpha chars
    review = re.sub('[^a-zA-Z\u00C0-\u00ff]', ' ', train['tokens'][i])
    # to lower-case 
    review = review.lower()
    # split into tokens, apply stemming and remove stop words
    review = ' '.join([stemmer.stem(w) for w in review.split() if not w in set(stopwords_list)])
    corpus.append(review)

print(corpus[:5])

['fact não apen frut ignor', 'hav hum jorn investig preocup aprofund contextual histór isenç relat preocup soc urg denunci muit peç real jorn', 'tud cómic fif', 'tod permit organiz faç total absurd sent', 'não faz rir cust poder']


## Obtaining Features and Classes

The next step is to obtain the features we will use to train our model.

For this, we will use TF-IDF with N-Grams

TODO: explore [TfidfVectorizer params](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
    
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(corpus).toarray()

print("(Number of samples, Number of features):", X.shape)

(Number of samples, Number of features): (16743, 69814)


In [6]:
y = train['label']

print(y.shape)

(16743,)


## Training classifiers

- *Naive Bayes*, the two most effective variants are [MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) and [ComplementNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html).
- *Logistic Regression*, through scikit-learn's [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) class.
- *Decision Tree*, through scikit-learn's [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) class. This model always assigns a probability of 1 to one of the classes.
- *Random Forest*, through scikit-learn's [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) class.
- *Support Vector Machines (SVM)*, through scikit-learn's [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) class. The SVM model also allows you to get probabilities, but for that you need to use the *probability=True* parameter setting in its constructor.
- *Perceptron*, through scikit-learn's [Perceptron](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html) class. This model does not allow you to get probabilities.
- *eXtreme Gradient Boosting*, through [XGBoost](https://xgboost.readthedocs.io/en/stable/).

TODO:
- Tune parameters
- [explore more CVs](https://scikit-learn.org/stable/modules/classes.html?highlight=model_selection#splitter-classes)

In [7]:
# Metrics
import sklearn.metrics as metrics
import time

# Cross Validation and Hyper Tuning
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV

# Classifiers
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
import xgboost as xgb

To train machine learning classifiers, we first split the data into training and test sets.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=0, stratify=y, shuffle=True)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print("\nLabel distribution in the training set:")
print(y_train.value_counts())

print("\nLabel distribution in the test set:")
print(y_test.value_counts())

(13394, 69814) (13394,)
(3349, 69814) (3349,)

Label distribution in the training set:
Value       6481
Fact        2930
Value(-)    2320
Value(+)    1129
Policy       534
Name: label, dtype: int64

Label distribution in the test set:
Value       1621
Fact         733
Value(-)     580
Value(+)     282
Policy       133
Name: label, dtype: int64


### Base line

In [9]:
def predict(clf):
    start = time.time()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    stop = time.time()

    # Metrics
    print("Elapsed time: %0.2fs" % (stop - start))
    print("\nConfusion matrix:\n", metrics.confusion_matrix(y_test, y_pred))
    print("Classification report:\n", metrics.classification_report(y_test, y_pred))

#### Naive Bayes

In [15]:
mnb = predict(MultinomialNB())

Elapsed time: 28.65s

Confusion matrix:
 [[  77    0  652    0    4]
 [   0    0  133    0    0]
 [  25    0 1590    0    6]
 [   5    0  277    0    0]
 [   7    0  555    0   18]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.68      0.11      0.18       733
      Policy       0.00      0.00      0.00       133
       Value       0.50      0.98      0.66      1621
    Value(+)       0.00      0.00      0.00       282
    Value(-)       0.64      0.03      0.06       580

    accuracy                           0.50      3349
   macro avg       0.36      0.22      0.18      3349
weighted avg       0.50      0.50      0.37      3349



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
cnb = predict(ComplementNB())

Elapsed time: 2.71s

Confusion matrix:
 [[ 292   26  287   55   73]
 [   3   63   54   10    3]
 [ 272   59 1003  101  186]
 [  36   14  101  110   21]
 [  64   21  196   14  285]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.44      0.40      0.42       733
      Policy       0.34      0.47      0.40       133
       Value       0.61      0.62      0.61      1621
    Value(+)       0.38      0.39      0.38       282
    Value(-)       0.50      0.49      0.50       580

    accuracy                           0.52      3349
   macro avg       0.45      0.47      0.46      3349
weighted avg       0.52      0.52      0.52      3349



#### SGD

In [17]:
sgd = predict(SGDClassifier(random_state=0, n_jobs=-1))

Elapsed time: 64.35s

Confusion matrix:
 [[ 225    1  432   25   50]
 [   1   47   77    6    2]
 [ 178   12 1279   40  112]
 [  36    4  156   72   14]
 [  43    1  314    0  222]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.47      0.31      0.37       733
      Policy       0.72      0.35      0.47       133
       Value       0.57      0.79      0.66      1621
    Value(+)       0.50      0.26      0.34       282
    Value(-)       0.56      0.38      0.45       580

    accuracy                           0.55      3349
   macro avg       0.56      0.42      0.46      3349
weighted avg       0.54      0.55      0.53      3349



#### Logistic Regression

In [18]:
lg = predict(LogisticRegression(random_state=0, n_jobs=-1))

Elapsed time: 4183.95s

Confusion matrix:
 [[ 185    0  512    7   29]
 [   0   11  121    0    1]
 [ 124    3 1436    5   53]
 [  23    1  231   25    2]
 [  33    1  416    0  130]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.51      0.25      0.34       733
      Policy       0.69      0.08      0.15       133
       Value       0.53      0.89      0.66      1621
    Value(+)       0.68      0.09      0.16       282
    Value(-)       0.60      0.22      0.33       580

    accuracy                           0.53      3349
   macro avg       0.60      0.31      0.33      3349
weighted avg       0.56      0.53      0.47      3349



#### SVC

In [20]:
svc = predict(SVC(random_state=0, max_iter=100))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Elapsed time: 928.35s

Confusion matrix:
 [[ 522    7  161   24   19]
 [  60   34   33    5    1]
 [1077   42  419   44   39]
 [ 193    6   66   12    5]
 [ 414   13  127    7   19]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.23      0.71      0.35       733
      Policy       0.33      0.26      0.29       133
       Value       0.52      0.26      0.35      1621
    Value(+)       0.13      0.04      0.06       282
    Value(-)       0.23      0.03      0.06       580

    accuracy                           0.30      3349
   macro avg       0.29      0.26      0.22      3349
weighted avg       0.37      0.30      0.27      3349



#### Decision Tree

In [15]:
dt = predict(DecisionTreeClassifier(random_state=0))

Elapsed time: 2798.89s

Confusion matrix:
 [[285   3 332  39  74]
 [  6  63  52   7   5]
 [327  44 998  85 167]
 [ 54   7 129  81  11]
 [ 94   3 277   9 197]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.37      0.39      0.38       733
      Policy       0.53      0.47      0.50       133
       Value       0.56      0.62      0.59      1621
    Value(+)       0.37      0.29      0.32       282
    Value(-)       0.43      0.34      0.38       580

    accuracy                           0.48      3349
   macro avg       0.45      0.42      0.43      3349
weighted avg       0.48      0.48      0.48      3349



#### Random Forest

In [16]:
rf = predict(RandomForestClassifier(random_state=0))

Elapsed time: 1182.63s

Confusion matrix:
 [[ 202    1  459   30   41]
 [   1   50   75    6    1]
 [ 168   19 1295   47   92]
 [  31    4  159   82    6]
 [  40    1  347    1  191]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.46      0.28      0.34       733
      Policy       0.67      0.38      0.48       133
       Value       0.55      0.80      0.65      1621
    Value(+)       0.49      0.29      0.37       282
    Value(-)       0.58      0.33      0.42       580

    accuracy                           0.54      3349
   macro avg       0.55      0.41      0.45      3349
weighted avg       0.54      0.54      0.51      3349



#### Perceptron

In [17]:
per = predict(Perceptron(random_state=0))

Elapsed time: 76.16s

Confusion matrix:
 [[ 192    2  461   28   50]
 [   2   52   72    5    2]
 [ 198   18 1241   45  119]
 [  27    4  174   70    7]
 [  47    1  330    1  201]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.41      0.26      0.32       733
      Policy       0.68      0.39      0.50       133
       Value       0.54      0.77      0.64      1621
    Value(+)       0.47      0.25      0.32       282
    Value(-)       0.53      0.35      0.42       580

    accuracy                           0.52      3349
   macro avg       0.53      0.40      0.44      3349
weighted avg       0.51      0.52      0.50      3349



### Parameter tuning

TODO [explore more scoring methods](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

In [10]:
def grid_search(clf, parameter_grid):
    cross_validation = StratifiedKFold(n_splits=5)

    grid_search = GridSearchCV(clf,
                               param_grid=parameter_grid,
                               scoring='accuracy',
                               cv=cross_validation,
                               verbose=4,
                               n_jobs=2,
                               refit=True)

    start = time.time()
    grid_search.fit(X_train, y_train)
    stop = time.time()
    print(f"Fit time: {stop - start}s")

    print("\nBest score:", grid_search.best_score_)
    print("Best parameters:", grid_search.best_params_)
    print("Best estimator:", grid_search.best_estimator_)
    
    best_model = grid_search.best_estimator_
    best_model_pred = best_model.predict(X_test)

    # Metrics
    print("\nConfusion matrix:\n", metrics.confusion_matrix(y_test, best_model_pred))
    print("Classification report:\n", metrics.classification_report(y_test, best_model_pred))

    return best_model

#### Naive Bayes

In [11]:
clf = MultinomialNB()

mnb = grid_search(clf, {})

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:  2.1min finished


Fit time: 133.56429934501648s

Best score: 0.49499782698447614
Best parameters: {}
Best estimator: MultinomialNB()

Confusion matrix:
 [[  77    0  652    0    4]
 [   0    0  133    0    0]
 [  25    0 1590    0    6]
 [   5    0  277    0    0]
 [   7    0  555    0   18]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.68      0.11      0.18       733
      Policy       0.00      0.00      0.00       133
       Value       0.50      0.98      0.66      1621
    Value(+)       0.00      0.00      0.00       282
    Value(-)       0.64      0.03      0.06       580

    accuracy                           0.50      3349
   macro avg       0.36      0.22      0.18      3349
weighted avg       0.50      0.50      0.37      3349



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
clf = ComplementNB()

cnb = grid_search(clf, {})

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:  1.8min finished


Fit time: 111.39218974113464s

Best score: 0.5205312472384304
Best parameters: {}
Best estimator: ComplementNB()

Confusion matrix:
 [[ 292   26  287   55   73]
 [   3   63   54   10    3]
 [ 272   59 1003  101  186]
 [  36   14  101  110   21]
 [  64   21  196   14  285]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.44      0.40      0.42       733
      Policy       0.34      0.47      0.40       133
       Value       0.61      0.62      0.61      1621
    Value(+)       0.38      0.39      0.38       282
    Value(-)       0.50      0.49      0.50       580

    accuracy                           0.52      3349
   macro avg       0.45      0.47      0.46      3349
weighted avg       0.52      0.52      0.52      3349



#### SGD

In [11]:
clf = SGDClassifier(random_state=0)

# The ‘log’ loss gives logistic regression, ‘perceptron’ is the linear loss used by the perceptron algorithm
parameter_grid= {'loss': ['hinge', 'perceptron'], 'penalty': ['elasticnet', 'l1', 'l2'], 'class_weight': [None, 'balanced']}

sgd = grid_search(clf, parameter_grid)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:   30.7s


MemoryError: Unable to allocate 5.57 GiB for an array with shape (10715, 69814) and data type float64

#### Logistic Regression

takes too long

In [ ]:
clf = LogisticRegression(random_state=0)

parameter_grid= {'solver': ['saga'], 'penalty': ['elasticnet', 'l1', 'l2'], 'l1_ratio': [0.5], 'class_weight': [None, 'balanced']}

lg = grid_search(clf, parameter_grid)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


#### SVC

In [ ]:
clf = SVC(random_state=0, max_iter=200)

parameter_grid= {}

svc = grid_search(clf, parameter_grid)

#### Decision Tree

In [15]:
clf = DecisionTreeClassifier(random_state=0)

parameter_grid= {}

dt = grid_search(clf, parameter_grid)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed: 106.8min finished


Fit time: 8592.86598443985s

Best score: 0.4885022528832529
Best parameters: {}
Best estimator: DecisionTreeClassifier(random_state=0)

Confusion matrix:
 [[285   3 332  39  74]
 [  6  63  52   7   5]
 [327  44 998  85 167]
 [ 54   7 129  81  11]
 [ 94   3 277   9 197]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.37      0.39      0.38       733
      Policy       0.53      0.47      0.50       133
       Value       0.56      0.62      0.59      1621
    Value(+)       0.37      0.29      0.32       282
    Value(-)       0.43      0.34      0.38       580

    accuracy                           0.48      3349
   macro avg       0.45      0.42      0.43      3349
weighted avg       0.48      0.48      0.48      3349



#### Random Forest

In [14]:
clf = RandomForestClassifier(random_state=0)

parameter_grid= {}

rf = grid_search(clf, parameter_grid)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed: 59.1min finished


Fit time: 4997.952701807022s

Best score: 0.5288928548629133
Best parameters: {}
Best estimator: RandomForestClassifier(random_state=0)

Confusion matrix:
 [[ 202    1  459   30   41]
 [   1   50   75    6    1]
 [ 168   19 1295   47   92]
 [  31    4  159   82    6]
 [  40    1  347    1  191]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.46      0.28      0.34       733
      Policy       0.67      0.38      0.48       133
       Value       0.55      0.80      0.65      1621
    Value(+)       0.49      0.29      0.37       282
    Value(-)       0.58      0.33      0.42       580

    accuracy                           0.54      3349
   macro avg       0.55      0.41      0.45      3349
weighted avg       0.54      0.54      0.51      3349



#### Perceptron

In [13]:
clf = Perceptron(random_state=0)

parameter_grid= {}

per = grid_search(clf, parameter_grid)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


MemoryError: Unable to allocate 5.57 GiB for an array with shape (10715, 69814) and data type float64

#### XGBoost

In [12]:
clf = xgb.XGBClassifier(random_state=0)

parameter_grid= {}

xgb = grid_search(clf, parameter_grid)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed: 19.4min finished
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:02:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fit time: 3000.7387795448303s

Best score: nan
Best parameters: {}
Best estimator: XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample

## Training models per annotator and using ensemble

Separating each annotator's judgment will make the task of training the model significantly easier and faster, since we are dealing with less data. At the end of the training, we will use ensemble methods, such as voting, to make a more accurate prediction, by using the predictions of each trained model to reach a consensus on a label.

In [8]:
X_annotated = pd.concat([train['annotator'], pd.DataFrame(X)], axis=1)

print(X_annotated)

MemoryError: Unable to allocate 8.71 GiB for an array with shape (69814, 16743) and data type float64

In [ ]:
X_A = X_annotated[X_annotated['annotator'] == 'A']
X_A = X_A.drop('annotator', 1)
y_A = train.loc[train['annotator'] == 'A', 'label']

X_B = X_annotated[X_annotated['annotator'] == 'B']
X_B = X_B.drop('annotator', 1)
y_B = train.loc[train['annotator'] == 'B', 'label']

X_C = X_annotated[X_annotated['annotator'] == 'C']
X_C = X_C.drop('annotator', 1)
y_C = train.loc[train['annotator'] == 'C', 'label']

X_D = X_annotated[X_annotated['annotator'] == 'D']
X_D = X_D.drop('annotator', 1)
y_D = train.loc[train['annotator'] == 'D', 'label']

print(X_A.shape)
print(y_A.shape)

In [ ]:
X_A_tr, X_A_te, y_A_tr, y_A_te = train_test_split(X_A, y_A, test_size = 0.20, random_state=0, stratify=y_A, shuffle=True)

X_B_tr, X_B_te, y_B_tr, y_B_te = train_test_split(X_B, y_B, test_size = 0.20, random_state=0, stratify=y_B, shuffle=True)

X_C_tr, X_C_te, y_C_tr, y_C_te = train_test_split(X_C, y_C, test_size = 0.20, random_state=0, stratify=y_C, shuffle=True)

X_D_tr, X_D_te, y_D_tr, y_D_te = train_test_split(X_D, y_D, test_size = 0.20, random_state=0, stratify=y_D, shuffle=True)

Now we can procede to use Voting in order to obtain a better prediction:

In [ ]:
from mlxtend.classifier import EnsembleVoteClassifier
import copy

clf_A = SGDClassifier(random_state=0, n_jobs=-1, loss='log')
clf_A.fit(X_A_tr, y_A_tr)

clf_B = SGDClassifier(random_state=0, n_jobs=-1, loss='log')
clf_B.fit(X_B_tr, y_B_tr)

clf_C = SGDClassifier(random_state=0, n_jobs=-1, loss='log')
clf_C.fit(X_C_tr, y_C_tr)

clf_D = SGDClassifier(random_state=0, n_jobs=-1, loss='log')
clf_D.fit(X_D_tr, y_D_tr)

clf_list = [clf_A, clf_B, clf_C, clf_D]

eclf = EnsembleVoteClassifier(clfs=clf_list, fit_base_estimators=False, voting='soft')
eclf.fit(X,y)
y_pred_vote = eclf.predict(X)
print(y_test)
print("Classification report:\n", metrics.classification_report(y, y_pred_vote))

## Removing minorities

To limit the amount of data we will work with, we will group data by tokens and labels, and count the max number of annotators per token-label pair. Next, we will remove the entries of the annotators that will be in a minority.

In [8]:
df_tmp = train.groupby(['tokens', 'label']).agg({
    'annotator': 'count'
}).reset_index()

train_no_duplicates = df_tmp.groupby(['tokens'], as_index=False).agg({'annotator': 'max', 'label': 'first'})
train_no_duplicates = train_no_duplicates.drop('annotator',1)
print(train_no_duplicates)

corpus = []
for i in range(0, train_no_duplicates['tokens'].size):
    # get review and remove non alpha chars
    review = re.sub('[^a-zA-Z\u00C0-\u00ff]', ' ', train_no_duplicates['tokens'][i])
    # to lower-case 
    review = review.lower()
    # split into tokens, apply stemming and remove stop words
    review = ' '.join([stemmer.stem(w) for w in review.split() if not w in set(stopwords_list)])
    corpus.append(review)
    
X = vectorizer.fit_transform(corpus).toarray()
y = train_no_duplicates['label']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=0, stratify=y, shuffle=True)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print("\nLabel distribution in the training set:")
print(y_train.value_counts())

print("\nLabel distribution in the test set:")
print(y_test.value_counts())

                                                  tokens  label
0      "porque é mais prático e, o que é pior, para t...   Fact
1      "revolução" é um termo frequente na literatura...   Fact
2      (In)felizmente, essas relações acabam, mais ce...   Fact
3                    (em 2017 foram quatro vezes maiores   Fact
4      (pelo que julgo poder deduzir) não está obriga...   Fact
...                                                  ...    ...
12003  “tratando-se de pares cujos elementos pertence...   Fact
12004  “um instituto universitário especializado com ...   Fact
12005  “um militar na polícia, nem consegue ser bom p...  Value
12006                           “Água pura, cristalina.”   Fact
12007      “área verde de enquadramento de espaço canal”  Value

[12008 rows x 2 columns]
(9606, 69814) (9606,)
(2402, 69814) (2402,)

Label distribution in the training set:
Value       4537
Fact        2549
Value(-)    1412
Value(+)     678
Policy       430
Name: label, dtype: int64

Label dis

### Naive Bayes

In [9]:
mnb = predict(MultinomialNB())

Elapsed time: 9.78s

Confusion matrix:
 [[  41    0  596    0    0]
 [   0    0  107    0    0]
 [  17    0 1117    0    1]
 [   2    0  168    0    0]
 [   3    0  350    0    0]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.65      0.06      0.12       637
      Policy       0.00      0.00      0.00       107
       Value       0.48      0.98      0.64      1135
    Value(+)       0.00      0.00      0.00       170
    Value(-)       0.00      0.00      0.00       353

    accuracy                           0.48      2402
   macro avg       0.23      0.21      0.15      2402
weighted avg       0.40      0.48      0.34      2402



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
cnb = predict(ComplementNB())

Elapsed time: 1.76s

Confusion matrix:
 [[243  15 309  22  48]
 [  7  15  76   4   5]
 [186  42 770  37 100]
 [ 27   4  95  36   8]
 [ 45   8 172  10 118]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.48      0.38      0.42       637
      Policy       0.18      0.14      0.16       107
       Value       0.54      0.68      0.60      1135
    Value(+)       0.33      0.21      0.26       170
    Value(-)       0.42      0.33      0.37       353

    accuracy                           0.49      2402
   macro avg       0.39      0.35      0.36      2402
weighted avg       0.48      0.49      0.48      2402



### SGD

In [11]:
sgd = predict(SGDClassifier(random_state=0, n_jobs=-1))

Elapsed time: 55.71s

Confusion matrix:
 [[242   0 365   8  22]
 [ 13  15  75   3   1]
 [174  16 872  22  51]
 [ 31   2 106  27   4]
 [ 53   2 228   1  69]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.47      0.38      0.42       637
      Policy       0.43      0.14      0.21       107
       Value       0.53      0.77      0.63      1135
    Value(+)       0.44      0.16      0.23       170
    Value(-)       0.47      0.20      0.28       353

    accuracy                           0.51      2402
   macro avg       0.47      0.33      0.35      2402
weighted avg       0.49      0.51      0.47      2402



### Logistic Regression

In [12]:
lg = predict(LogisticRegression(random_state=0, n_jobs=-1))

MemoryError: Unable to allocate 5.00 GiB for an array with shape (9606, 69814) and data type float64

### SVC

In [ ]:
svc = predict(SVC(random_state=0, max_iter=100))

### Decision Tree

In [ ]:
dt = predict(DecisionTreeClassifier(random_state=0))